# Unsupervised learning

L'objectif de ce TP ce TP est de tester et de comprendre les différentes méthodes de ML non-supervisé. 
Dans ce TP, vous devez remplacer les ... par le code adéquat.

- [Dimensionnality reduction](#Dimensionality-reduction)
  - [PCA](#PCA)
  - [Manifold Learning](#Manifold-Learning)
- [Clustering](#Clustering)
  - [KMeans](#KMeans)
  - [DBSCAN](#DBSCAN)
  - [GMM](#GMM)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Réduction de dimension

### Creation d'un jeu de donnné

In [ ]:
# Creation d'un jeu de donnees avec deux axes corrélés
np.random.seed(42)
X = (np.random.rand(2, 2) @ np.random.randn(2, 200)).T
plt.scatter(X[:, 0], X[:, 1]);

### PCA
**Principal Component Analysis**

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
# Crée un PCA avec 2 composantes et l'utilise pour fitter X
# Vous pouvez ensuite accéder aux composantes principales avec pca.components_ et à la variance avec pca.explained_variance_
pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
print("Composantes principales : ")
print(pca.components_)

print("Variance : ")
print(pca.explained_variance_)

#### Visualisation des composants

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax if ax is not None else plt.gca()
    arrowprops = dict(arrowstyle='->', color='k', lw=2, shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

In [ ]:
# A l'aide la la fonction draw_vector, ont peux les composantes principales sur le plot précédent
plt.scatter(X[:, 0], X[:, 1])
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)

#### Transformation des données

In [ ]:
#Utiliser le PCA pour transformer X selon les composantes principales
X_trans = pca.fit_transform(X)
plt.scatter(X_trans[:, 0], X_trans[:, 1])

#### Réduction de dimension

In [ ]:
# En utilisant le PCA avec une dimension inférieur à celle de X, on peut réduire la dimension des données d'entrée
pca = PCA(n_components=1)
X_trans = pca.fit_transform(X)
#Utiliser ensuite la fonction inverse_transform permet pour revenir dans l'espace d'origine
X2 = pca.inverse_transform(X_trans)


In [ ]:
plt.scatter(X[:, 0], X[:, 1], label='données initiales')
plt.scatter(X2[:, 0], X2[:, 1], label='données après PCA')
plt.legend(frameon=False, loc=4)

### Manifold Learning

Nous allons maintenant essayer de réduire la dimension de données de chiffres manuscrits à l'aide du PCA et de manifolfd learning 

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(10, 6), subplot_kw={'xticks': [], 'yticks': []})
for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap='binary_r')
plt.grid(False)


#### PCA

In [ ]:
# Pour commencer utiliser le PCA pour réduire la dimension des données (digits.data)

projected_data = PCA(2).fit_transform(digits.data)

plt.figure(figsize=(10,6));
plt.scatter(projected_data[:, 0], projected_data[:, 1], c=digits.target, edgecolor='none', alpha=0.5, cmap=plt.colormaps['cubehelix'])
plt.colorbar()

#### t-SNE
Comme nous l'avons vu en cours le PCA ne fonction que pour les donnée séparrable linéairent. Nous allons donc essayer le t-distributed stochastic neighbor embedding pour séparer les chiffres

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# Utiliser le TSNE pour réduire la dimension des données, la syntaxe est similaire à celle du PCA

projected_data = TSNE(n_components=2).fit_transform(digits.data)
plt.scatter(projected_data[:, 0], projected_data[:, 1], c=digits.target, edgecolor='none', alpha=0.5, cmap=plt.cm.get_cmap('cubehelix', 10))
plt.colorbar()

## Clustering

- K-means, DBSCAN

    https://scikit-learn.org/stable/modules/clustering.html
    
- Gaussian mixture

    https://scikit-learn.org/stable/modules/mixture.html

#### Donnée

In [ ]:
from sklearn.datasets import make_blobs
np.random.seed(44)
X, labels = make_blobs(n_samples=100, centers=10, cluster_std=0.3)

In [ ]:
def plot_blobs(X, y=None, n=3):
    fig, ax = plt.subplots(figsize=(10, 8))
    if y is None:
        ax.scatter(X[:, 0], X[:, 1])
        ax.set_title("Raw data", fontsize=14)
    else:
        im = ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.get_cmap('viridis', n))
        ax.set_title("Labeled data", fontsize=14)

In [ ]:
plot_blobs(X)

In [ ]:
plot_blobs(X, labels, 10)

### KMeans

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Utiliser KMeans pour prédire les labels des données X

kmeans = KMeans(n_clusters=10, init="k-means++", n_init=5).fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
plot_blobs(X, y_kmeans, 10)

### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
#Utiliser maintenant DBSCAN pour prédire les labels des données X 
dbscan = DBSCAN(eps=0.8, min_samples=3)
y_dbscan = dbscan.fit_predict(X)

print(f"n_clusters = {len(set(dbscan.labels_))}")

In [ ]:
# Puis plotter le résultat
plot_blobs(X, y_dbscan, len(set(dbscan.labels_)))

## GMM
**Gaussian Mixture Models**

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
# Pour finir essayer la même chose avec un GaussianMixture

gmm = GaussianMixture(n_components=10).fit(X)
y_gmm = gmm.predict(X)

In [ ]:
plot_blobs(X, y_gmm, 10)

### Application à des jeux de donnée plus complexe

Les donnée utilisé jusque ici étais extrement simple, il est donc normal qu'on trouve les bons clusters.
Essayons maintenant avec trois jeux de données plus complexes. A vous de determiner la solution la plus approprié à chaque.

In [ ]:
# Blob avec des corrélations
np.random.seed(40)
Xi, labelsi = make_blobs(n_samples=300, centers=5, cluster_std=1.5)
X2 = Xi @ np.random.rand(2, 2)
plot_blobs(X2, labelsi, 5)

In [ ]:
gmm2 = GaussianMixture(n_components=5).fit(X2)
y_gmm2 = gmm2.predict(X3)
plot_blobs(X2, y_gmm2, 5)

In [ ]:
# Nombre de blob inconue
np.random.seed(22)
X3, labels3 = make_blobs(n_samples=200, centers=int(10+np.random.rand()*20), cluster_std=0.3)
plot_blobs(X3, labels3, labels3.max()+1)

In [ ]:
print(f"n_clusters original = {labels3.max()+1}")

#Utiliser maintenant DBSCAN pour prédire les labels des données X 
dbscan2 = DBSCAN(eps=0.4, min_samples=5)
y_dbscan2 = dbscan3.fit_predict(X4)

print(f"n_clusters = {len(set(dbscan2.labels_))-1}")

plot_blobs(X3, y_dbscan2, len(set(dbscan2.labels_)))

In [ ]:
# Blobs avec des variances différentes supposées
np.random.seed(10)
Xb, labelsb = make_blobs(n_samples=300, centers=3, cluster_std=5)
# Ajout de blobs sulémentaires
X4 = np.vstack([X, Xb])
labels4 = np.hstack([labels, labelsb+labels.max()+1])
plot_blobs(X4, labels4, 13)

In [ ]:
# DBSCAN pour extraire les plus petit blobs
dbscan3 = DBSCAN(eps=0.5, min_samples=5)
y_dbscan3 = dbscan3.fit_predict(X4)

print(f"n_clusters = {len(set(dbscan3.labels_))-1}")
plot_blobs(X4, y_dbscan3, len(set(dbscan3.labels_)))

# Le bruit contient les trois plus grand blobs
X_noise = X4[y_dbscan3==-1]

gmm3 = GaussianMixture(n_components=3).fit(X_noise)
y_gmm3 = gmm3.predict(X_noise)
plot_blobs(X_noise, y_gmm3, 3)

# On remplace les labels du bruit par ceux du GMM
y_dbscan3[y_dbscan3==-1] = y_gmm3

plot_blobs(X4, y_dbscan3, len(set(y_dbscan3)))
